# Machine interpreter

## Instructions

After a good day sightseeing, it's back to the hotel and time for some refreshment. Unfortunately, the minibar in your room refuses to open. It's been hacked, with some ransomware! You'll need to provide the correct unlock code so you can get a nice, cold drink.

You could pay a large chunk of bitcoin, or you could defeat the ransomware some other way. 

You quickly find the schematics of the minibar lock. It's a fairly simple machine. It has four registers, `a`, `b`, `c`, `d`, and a special purpose `pc` register for the program counter. Each register can hold a 64-bit value (far bigger than any number in the programs you'll be running). You can assume all registers hold zero when the program starts.

On each clock tick, the machine executes the instruction pointed to by the `pc`, then increments `pc`. The machine halts when the machine tries to read from a location beyond the end of the program.

The machine has only a few instructions. They're listed by handy mnemonics:

| Instruction | Description |
|:------------|:------------|
| `inc r`     | increment contents of register `r` |
| `dec r`     | decrement contents of register `r` |
| `set r i`   | set contents of register `r` to literal value `i` |
| `cpy r s`   | copy contents of register `r` into register `s` | 
| `jmp i`     | jump to instruction `i` places forward |
| `jpz r i`   | jump to instruction `i` places forward if<br>register `r` contains zero, otherwise continue to next instruction |

The `jmp` and `jpz` instructions jump relative to the current instruction, overriding the normal change in `pc`. `jmp -1` would jump back to the previous instruction; `jpz a 2` would skip the next instruction if register `a` contains zero.

Before you start execution of a program, you can set the values of some registers.

For example, this program multiplies the values in the a and b registers, leaving the result in the c register:

```
set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
set d 0
```

# Part 1

You think you've worked out how to generate the code wanted by the ransomware. The program is given in [07-program](07-program.txt), one instruction per line. 

Starting with register `a` holding 7, and all other registers holding zero, what does register `a` contain when the program finishes?

It seems your guess of 7 as the starting value was wrong.

# Part 2

The program is still given in [07-program.txt](07-program.txt), one instruction per line. 

Starting with register `a` holding 937, and all other registers and memory locations holding zero, what does register `a` contain when the program finishes?

# Worked example solution: parts 1 and 2
This follows a very similar idea to [Day 5](../05-display-board/display-board-solution.ipynb): we're building a virtual machine and applying a set of instructions to it.

The virtual machine is, if anything, simpler than the laser display board from day 5. There are just four registers, the special `pc` register, and the list of instructions. 

Again, as with the laser display board, each operation is defined as Python function, which takes as arguments the instruction's arguments and the machine being operated on. The operations update the machine in-place.

One decision is over what counts as an 'instruction'. I've taken the view that what's stored in the machine should be as close to what's exectuted as possible. Therefore, each instruction is a pair (2-tuple) of the function that performs that instruction. The first element is the Python function that performs the operation; the second is a list of arguments, either register names or literal number values. See the `run()` procedure for how it's used.

## Parsing instructions
Another change from the laser display board is how the instructions are parsed. In the same way as before, I use a dispatch table (called `instruction_table`) to relate the instruction names to the Python procedures that perform them. But I also have a the `numeric_args_table` which specifies which arguments to which instructions should be processed as numbers. The `parse` procedure reads both of those and converts the instruction text into the correct Python representation.

In [3]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'd': 0,
            'instructions': []}

In [4]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [5]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [6]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [7]:
def jmp(addr, machine):
    machine['pc'] += addr

In [8]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [9]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [10]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [11]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [12]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [13]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

Note that I've added an extra step in the program parsing stage, which is dealing with labels. When you're writing programs like this, it's very easy to lose track of where each instruction is and how large the jumps should be. To get around that, I take a leaf from standard assembly programming of allowing user-defined labels for positions. That allows me to write the example multiplication program like this:

```
      set c 0
      cpy a d
loop: jpz b end      
      dec b
      cpy d a
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  set d 0
```

where the words at the start of the line are the labels and all the jumps move to particular labels. This is a bit more readable, and also means I can add and remove instructions without having to change all the jump instructions as well.

The `replace_labels` function makes two passes through the program listing. The first pass finds all the labels and stores their absolute positions. The second pass replaces each label in an instruction with the relative position from the instruction number to the label's position.

In [14]:
def unlabel_listing(listing):
    labelled_instructions = [i.strip() for i in listing.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
    return replace_labels(labelled_instructions)    

In [15]:
def program_from_listing(listing, machine):
    instructions = unlabel_listing(listing)
    program_from_instructions(instructions, machine)

In [16]:
def replace_labels(listing):
    locations = {}
    for n, i in enumerate(listing):
        if ':' in i:
            locations[i.split(':')[0]] = n

    unlabelled_listing = []
    for n, i in enumerate(listing):
        instr = i.split()
        if ':' in i:
            instr = i.split(':')[1].split()
        else:
            instr = i.split()
        terms = []
        for term in instr:
            if term in locations:
                terms += [str(locations[term] - n)]
            else:
                terms += [term]
        transformed_instr = ' '.join(terms)
        unlabelled_listing += [transformed_instr]
        
    return unlabelled_listing    

Note the 'splat operator' trick, which unpacks a list of items into positional parameters of the function. It has the effect of converting
```
cpy(['a', 'd'], machine)
```
into
```
cpy('a', 'd', machine)
```
which is the form I want. 

Another way of doing this could be to have the individual instruction-implementing procedures take a list of parameters, but I think this is clearer to understand.

In [17]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [18]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [22]:
program = open('07-program.txt').read()
show_machine(execute(program, initial_state={'a': 7}))

'a: 52, b: 0, c: 0, d: 0, pc: 48'

In [23]:
program = open('07-program.txt').read()
show_machine(execute(program, initial_state={'a': 937}))

'a: 250504, b: 0, c: 0, d: 0, pc: 48'